## Sfincs results: maximum water depth

The example used in this notebook is based on a regular SFINCS model, i.e. no subgrid tables are used. In the absence of the subgrid tables, SFINCS computes the water depth by simply substracting the bed levels from the water levels. The (maximum) water depth **h(max)** is stored in the NetCDF output (*sfincs_map.nc*). 

How to derive maximum water depths for a model including subgrid tables is explained [later](#determine-maximum-water-depth-for-a-subgrid-model) in this notebook.

In [ ]:
from os.path import join
import matplotlib.pyplot as plt
from hydromt_sfincs import SfincsModel, utils

### Read model results

The model results in sfincs_map.nc are saved as in a staggered grid format, see [SGRID convention](https://publicwiki.deltares.nl/display/NETCDF/Deltares+proposal+for+Staggered+Grid+data+model+(SGRID)). Here we show how to retrieve the face values and translate the dimensions from node indices (m, n) to (x, y) coordinates in order to plot the results on a map.

In [ ]:
sfincs_root = "sfincs_compound"  # (relative) path to sfincs root
mod = SfincsModel(sfincs_root, mode="r")

In [ ]:
# we can simply read the model results (sfincs_map.nc and sfincs_his.nc) using the read_results method
mod.read_results()
# the following variables have been found
list(mod.results.keys())

In [ ]:
# plot the model layout
fig, ax = mod.plot_basemap(fn_out=None, bmap="sat", figsize=(11, 7))

### Write maximum waterdepth to geotiff file

In [ ]:
# write hmax to <mod.root>/gis/hmax.tif
mod.write_raster("results.hmax", compress="LZW")

# this is identical to the following:
# hmax = mod.results['hmax']
# hmax = hmax.reindex(y=list(reversed(hmax['y'].values))) # change orientation to N -> S
# hmax.raster.to_raster(join(mod.root, 'gis', 'hmax.tif'), compress='LZW')

### Plot maximum water depth with surface water mask 

First we mask the water depth based on a map of permanent water to get the flood extent. 
The mask is calculated from the Global Surface Water Occurence (GSWO) dataset. 

In [ ]:
# read global surface water occurance (GSWO) data to mask permanent water
mod.data_catalog.from_artifacts()
print(mod.data_catalog["gswo"])
gswo = mod.data_catalog.get_rasterdataset("gswo", geom=mod.region, buffer=10)

# NOTE to read data for a different region than Northen Italy add this data to the data catalog:
# mod.data_catalog.from_yml('/path/to/data_catalog.yml')

# permanent water where water occurence > 5%
gswo_mask = gswo.raster.reproject_like(mod.grid, method="max") <= 5

In [ ]:
hmin = 0.05  # minimum flood depth [m] to plot

# hmax is computed by SFINCS and read-in from the sfincs_map.nc file
da_hmax = mod.results["hmax"]  

# get overland flood depth with GSWO and set minimum flood depth
da_hmax_fld = da_hmax.where(gswo_mask).where(da_hmax > hmin)

# update attributes for colorbar label later
da_hmax.attrs.update(long_name="flood depth", unit="m")

Here we plot the maximum water depth on top of the `plot_basemap` method to also include the locations of discharge source points and observation gauge locations.

In [ ]:
# create hmax plot and save to mod.root/figs/hmax.png
fig, ax = mod.plot_basemap(
    fn_out=None,
    variable="", # no variable to plot, only basemap
    plot_bounds=False, 
    bmap="sat",
    zoomlevel=12,
    figsize=(11, 7),
)
# plot overland flooding based on gswo mask and minimum flood depth
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
cax_fld = da_hmax_fld.plot(
    x="xc", y="yc",
    ax=ax,
    vmin=0, vmax=3.0,
    cmap=plt.cm.viridis,
    cbar_kwargs=cbar_kwargs
)

ax.set_title(f"SFINCS maximum water depth")
# plt.savefig(join(mod.root, 'figs', 'hmax.png'), dpi=225, bbox_inches="tight")

### Determine maximum water depth for a subgrid model

When subgrid tables are used, these are based on elevation data of a resolution higher than the resolution of the computational grid. The bed levels stored in the NetCDF output (*zb* in *sfincs_map.nc*) are the minimum bed levels from the subgrid tables of each computational cell. Therefore, to properly derive water depths, the higher resolution elevation data should be used instead of simply using the bed levels from the model output (this would result in an overestimation of flood extents). The process of interpolating the water levels onto the higher-resolution elevation data is called downscaling.

The downscaling of the floodmap roughly includees the following steps:

1. Select your high-resolution elevation dataset
2. Retrieve the maximum water levels from your model
3. Determine if and how you want to mask your floodmap
4. Write a geotiff of your downscaled floodmap
5. Plot your downscaled floodmap here, or in QGIS

In [ ]:
# select the example model
sfincs_root = "sfincs_compound"  # (relative) path to sfincs root
mod = SfincsModel(sfincs_root, mode="r")

In the example, we used *GEBCO* (~450m resolution) and *MERIT Hydro* (~90m resolution) to create a model on 50 meters resolution. In this case, including subgrid tables didnt't add much information to the model. To still illustrate the process of downscaling, we will use the *dep.tif* (in the gis folder) which has 50m resolution. 

When creating a subgrid with `setup_subgrid`, you can easily create a geotiff on the subgrid resolution with the argument `write_dep_tif.

In [ ]:
# first we are going to select our highest-resolution elevation dataset
depfile = join(sfincs_root, "gis", "dep.tif")

# with the depfile on subgrid resolution this would be:
# depfile = join(sfincs_root, "subgrid", "dep_subgrid.tif")

dep = mod.data_catalog.get_rasterdataset(depfile)

In [ ]:
# secondly we are reading in the model results
mod.read_results()

# now assuming we have a subgrid model, we don't have hmax available, so we are using zsmax (maximum water levels)
zsmax = mod.results["zsmax"]

# compute the maximum over all time steps
zsmax = zsmax.max(dim='timemax')

In [ ]:
# Thirdly, we determine the masking of the floodmap

# we could use the GSWO dataset to mask permanent water in a similar way as above
# NOTE: this is masking the water levels on the computational grid resolution

# alternatively we could use a geodataframe (e.g. OpenStreetMap landareas) to mask water bodies
# NOTE: small rivers are not masked with this geodataframe
gdf_osm = mod.data_catalog.get_geodataframe("osm_landareas")

# and again, we can use a threshold to mask minimum flood depth
hmin = 0.05

In [ ]:
# Fourthly, we downscale the floodmap and save to <mod.root>/floodmap.tif
floodmap_fn = join(sfincs_root, "floodmap.tif")
hmax = utils.downscale_floodmap(zsmax=zsmax, dep=dep, hmin=hmin, gdf_mask=gdf_osm, floodmap_fn=floodmap_fn)

In [ ]:
# Lastly, we create a basemap plot with hmax on top
fig, ax = mod.plot_basemap(
    fn_out=None,
    variable="", # no variable to plot, only basemap
    plot_bounds=False, 
    bmap="sat",
    zoomlevel=12,
    figsize=(11, 7),
)

cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
cax_fld = hmax.plot(
    x="xc", y="yc",
    ax=ax,
    vmin=0, vmax=3.0,
    cmap=plt.cm.viridis,
    cbar_kwargs=cbar_kwargs,
)

ax.set_title(f"SFINCS maximum water depth")
# plt.savefig(join(mod.root, 'figs', 'hmax.png'), dpi=225, bbox_inches="tight")